In [1]:
import numpy as np
import random
import torch as T
import torch.nn as nn
import torch.nn.functional as TNF
import pyro as P
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as PD
from pyro.nn import PyroModule, PyroParam, PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from matplotlib import pyplot as plt
from made import MADE


class simpleNNNormal(nn.Module):
    def __init__(self, input_size=1, hidden=16, out_size=1):
        super().__init__()
        self.hidden_layer = nn.Linear(input_size, hidden)
        self.loc_layer = nn.Linear(hidden, out_size)
        self.std_layer = nn.Linear(hidden, out_size)
        
    def forward(self, x_list):
        x = TNF.relu(self.hidden_layer(x_list))
        loc = self.loc_layer(x)
        std = self.std_layer(x)
        return loc, std

class simpleNNHidden(nn.Module):
    def __init__(self, input_size=1, hidden=16, out_size=1):
        super().__init__()
        self.hidden_layer = nn.Linear(input_size, hidden)
        self.out_layer = nn.Linear(hidden, out_size)

    def forward(self, x_list):
        x = TNF.relu(self.hidden_layer(x_list))
        return self.out_layer(x)


hid_size = 3

nn_model_core_hidden = simpleNNHidden(out_size=hid_size)
nn_q__i = simpleNNHidden(input_size=hid_size)
nn_xp__i = simpleNNNormal(input_size=hid_size)
nn_y2__i = simpleNNNormal(input_size=hid_size + 1)
nn_y1__i =  simpleNNNormal(input_size=hid_size + 1)
nn_core_hidden = simpleNNHidden(input_size=hid_size, out_size=hid_size)
#nn_core_hidden = simpleNNHidden(input_size=hid_size + 1, out_size=hid_size)

def core_guide(hidden, x, i=0):
    q__i_prob, = nn_q__i(hidden)
    q = P.sample(f'q_{i}', PD.Bernoulli(T.sigmoid(q__i_prob)))
    #cond = P.deterministic(f"cond_{i}", q < 0.5)
    #if cond:
    if q < 0.5:
        return x
    else:
        xp__i_mean, xp__i_std = nn_xp__i(hidden)
        xp = P.sample(f'xp_{i}', PD.Normal(xp__i_mean, TNF.softplus(xp__i_std)))
        y2__i_mean, y2__i_std = nn_y2__i(T.cat([hidden, T.tensor([xp])]))
        y2 = P.sample(f'y2_{i}', PD.Normal(y2__i_mean, TNF.softplus(y2__i_std)))
        y1__i_mean, y1__i_std = nn_y1__i(T.cat([hidden, T.tensor([y2])]))
        y1 = P.sample(f'y1_{i}', PD.Normal(y1__i_mean, TNF.softplus(y1__i_std)))
        return core_guide(nn_core_hidden(hidden), xp, i + 1) # nn_core_hidden should take x
        #return core_guide(nn_core_hidden(T.cat([hidden, T.tensor([q__i_prob])])), xp, i + 1) # nn_core_hidden should take x

def guide(obs):
    P.module("guidenn_1", nn_model_core_hidden)
    P.module("guidenn_2", nn_q__i)
    P.module("guidenn_3", nn_xp__i)
    P.module("guidenn_4", nn_y2__i)
    P.module("guidenn_5", nn_y1__i)
    P.module("guidenn_6", nn_core_hidden)
    r = core_guide(nn_model_core_hidden(T.tensor([obs])), 0.0, i=0)

def core_model(x, i=0):
    q = P. sample (f"q_{i}", PD. Bernoulli (T. tensor (0.5)))
    if q < 0.5:
        return x
    else :
        y1 = P.sample (f'y1_{i}', PD.Normal(x, 0.1))
        y2 = P.sample (f'y2_{i}', PD.Normal(y1 , 0.1))
        xp = P.sample (f'xp_{i}', PD.Normal(y2 , 0.1))
        return core_model(xp , i+1) + x

def model(obs):
    r = core_model(0.0,i=0)
    return P.sample("r", PD.Normal(r ,0.01) , obs=obs)



In [ ]:
def gen_data(n=100):
    d = []
    for _ in range(n):
        d.append(model(obs=None))
    return d

data = gen_data()
P.clear_param_store()
num_data = len(data)
adam_params = {"lr": 0.001, "betas": (0.95, 0.999)}
optimizer = Adam(adam_params)

svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

n_steps = 200
log_interval = 10
# do gradient steps
loss = 0
loss_track = []
for step in range(1, n_steps + 1):
    imme_loss = 0
    for obs in data:
        imme_loss += svi.step(obs) / num_data
    loss_track.append(imme_loss)
    loss += imme_loss / log_interval
    
    if step % log_interval == 0:
        print("[Step {}/{}] Immediate Loss: {} Accumlated Loss: {}".format(step, n_steps, imme_loss, loss))
        loss = 0

plt.plot(loss_track)
plt.title("Loss")
plt.show()

[Step 10/200] Immediate Loss: 451.4602196265757 Accumlated Loss: 793.7536484837383
[Step 20/200] Immediate Loss: 229.44013524167252 Accumlated Loss: 358.7724746289477
[Step 30/200] Immediate Loss: 99.74989255601537 Accumlated Loss: 199.42344435065334
[Step 40/200] Immediate Loss: 56.224246747265575 Accumlated Loss: 141.6763683039278
[Step 50/200] Immediate Loss: 148.46230420928856 Accumlated Loss: 135.958261369531
[Step 60/200] Immediate Loss: 129.8268276126356 Accumlated Loss: 60.237155390869496
[Step 70/200] Immediate Loss: 26.011371933324412 Accumlated Loss: 30.07615272825404
[Step 80/200] Immediate Loss: 11.038791142533391 Accumlated Loss: 19.472201776273987
[Step 90/200] Immediate Loss: 6.9147539241502285 Accumlated Loss: 7.6087177703324445
[Step 100/200] Immediate Loss: 3.6030296424641994 Accumlated Loss: 5.132802795604141
[Step 110/200] Immediate Loss: 4.247810385392296 Accumlated Loss: 10.262509550340054
[Step 120/200] Immediate Loss: 3.620147859739383 Accumlated Loss: 3.796022